In [ ]:
import numpy as np
import time
from numba import cuda
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

### Xét class **`Tree`**

**Song song hóa hàm `find_best_split()`**

**Mô tả hàm**: `find_best_split()` là hàm tìm giá trị split sao cho gain của chúng khi rẽ nhánh đạt giá trị lớn nhất => đây là giá trị split tốt nhất dùng để rẽ nhánh.

**Ý tưởng tuần tự**: Vét cạn tất cả trường hợp: Xét feature thứ i, tìm từng giá trị split theo feature i, tính gain và so sánh với `best_gain`, lặp lại đến hết và sau cùng lưu vị trí feature cùng với giá trị split và best_gain.

**Ý tưởng song song**: Chia làm 2 hàm con:
- `compute_split_value()`: tính tất cả giá trị split value, mỗi thread sẽ đảm nhiệm tính một giá trị split
- `compute_similarity()`: sau khi xác định được mảng giá trị split, với mỗi giá trị, ta tiến hành rẽ nhánh và tính tổng similarity của nhánh bên trái và nhánh bên phải.
  
Kết hợp với `np.argmax()` của thư viện numpy để tìm được giá trị split tốt nhất. Tương tự như ý tưởng tuần tự, ta lưu vị trí feature cùng với giá trị split và best_gain.

In [ ]:
''' Hàm tính tất cả giá trị split value. Mỗi thread sẽ tìm split value của 2 giá trị liên tiếp
    Nếu giá trị tại feature thứ i của mẫu A và B bằng nhau -> trả giá trị nan
    Nếu giá trị tại feature thứ i của mẫu A và B khác nhau -> trả giá trị mean của 2 feature thuộc 2 mẫu
    Tham số
        input: ma trận dữ liệu X
        output: ma trận split value

    Trước khi đưa vào input sẽ được sort axis = 0 -> sort lại theo từng feature để dễ dàng tìm split value
'''

@cuda.jit
def compute_split_value(input, output):
    c, r = cuda.grid(2)
    if r < output.shape[0] and c < output.shape[1]:
        if input[r, c] != input[r + 1, c]:
            split = (input[r, c] + input[r + 1, c]) / 2
            output[r, c] = split
        else:
            output[r, c] = np.nan